In [2]:
from openai import OpenAI
import os
from typing_extensions import override
from openai import AssistantEventHandler
import random
import re
from config import name_of_book, full_book_txt, client, model_to_use, backup_model

model_to_use="gpt-3.5-turbo-0125"
backup_model="gpt-4-turbo"
count = 1


In [2]:
chunk_directory=os.path.join(name_of_book,"text_chunks")

with open(full_book_txt, 'r', encoding='utf-8') as file:
    remaining_text = full_text = file.read()

if os.path.exists(chunk_directory):
    files = [f for f in os.listdir(chunk_directory) if os.path.isfile(os.path.join(chunk_directory, f))]
    # Sort files alphabetically
    files.sort()
    # Return the last file if there are any files
    if files:
        filepath = os.path.join(chunk_directory, files[-1])

        name_of_file = files[-1]
        without_the_other_stuff = int(name_of_file.replace(chunk_directory,"").replace(".txt",""))
        count += without_the_other_stuff

        with open(filepath, 'r', encoding='utf-8') as file:
            finished_text = file.read()

            length_to_cut = remaining_text.find(finished_text)
            length_to_cut += len(finished_text)

            remaining_text=remaining_text[length_to_cut:]

# print(remaining_text)
percent_complete = 100 - (len(remaining_text)/len(full_text))*100
percent_complete = round(percent_complete, 2)
print(percent_complete)

18.42


In [3]:
import os

def write_text_to_file(folder, filename, text_to_write):
    """
    Writes text to a file within a specified folder. Creates the folder if it does not exist.
    
    Args:
    folder (str): The directory to store the file.
    filename (str): The name of the file to create.
    text_to_write (str): The text to write to the file.
    """
    # Ensure the folder exists
    os.makedirs(folder, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder, f"{filename}.txt")
    
    # Write the text to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text_to_write)
    print(f"Text has been written to {file_path}")


In [4]:
def number_to_four_digit_string(num):
    digit_string = str(num)
    remaining_length = 4 - len(digit_string)
    return remaining_length * "0" + digit_string

In [5]:
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.


class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 

In [6]:
assistant = client.beta.assistants.create(
    name="Literature Chunker",
    instructions="You are meant to divide text into segments based on natural breaks in the narrative, ensuring chunks can stand alone with its accompanying illustration",
    tools=[],
    model=model_to_use,
)

assistant2 = client.beta.assistants.create(
    name="Literature Chunker",
    instructions="You are meant to divide text into segments based on natural breaks in the narrative, ensuring chunks can stand alone with its accompanying illustration",
    tools=[],
    model=backup_model,
)

def limit_context_window(upgrade=False):

    thread = client.beta.threads.create()

    prompt = "You are meant to divide text into segments based on natural breaks in the narrative, ensuring chunks can stand alone with its accompanying illustration. Do not give a break that causes a sentence to be split in half. A few examples are: "
    prompt += '''\n"No, it mustn't be so any more; we must not part like that! what are two evenings?"'''
    prompt += '\n"I bless my luck for the excellent knotted stick, which happened on that occasion to be in my right hand."'
    prompt += '\n"And timid as I was with women, yet this was such a moment!"'

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id if upgrade == False else assistant.id,
        instructions="Please follow the user's instructions and acknowlege that you understand.",
        # event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        limit=1

    )

    return thread

In [7]:
def get_length_to_cut(literature, count=0):
    prompt = "Okay, find the first natural break in this snippet, where you think it would be appropriate to add an accompanying illustration."
    prompt += "The intention is that the chunks can stand alone with its accompanying illustration."
    prompt += "Tell me where the break is by telling me the last ten or so words from the snippet that marks the break."
    prompt += "**Enclose the last sentence** between two triple hashes, like ###At this point Nastenka stopped and began laughing.###"
    # prompt += "The last few bits that marks the break, should have the exact punctuation from the text."
    prompt += "**Make sure that you do not cut off a word or a sentence**."
    prompt += "Do not capitalize anything unless it was already capitalized. Do not lowercase anything unless it was already lowercased."
    prompt += "Do not give me multiple breaks."
    prompt += "Take all the time you need to get it done right the first time. It's better to be late than incorrect."
    prompt += "Please, this is for spreading literature to the masses."

    if count > 20: prompt += f"This is not the first time I have asked variants of you for it, and it's still not working. Please look for the {round(count/10) + 1}th best solution"

    thread = limit_context_window(upgrade=False if count < 10 else True)

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt+"'''" + literature + "'''"
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please follow the user's instructions.",
        # event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        limit=1

    )
    # type(messages)
    length_to_cut = 0
    for message in messages:
        raw_response = message.content[0].text.value

        pattern = r'###(.*?)###'

        # Use re.search to grab the first occurrence
        match = re.search(pattern, raw_response)

        if not match:
            if count % 40 == 0 and count != 0: print("\n")
            print("🙅",end="")
            return get_length_to_cut(literature, count=count+1)
        text_that_got_returned = match.group(1)

        if len(text_that_got_returned) < 30:
            if count % 40 == 0 and count != 0: print("\n")
            print("🤏",end="") 
            return get_length_to_cut(literature, count=count+1)
        if text_that_got_returned[-1] not in [".","!","?",'"']:
            if count % 40 == 0 and count != 0: print("\n")
            print("🔚",end="")
            return get_length_to_cut(literature, count=count+1)
        if text_that_got_returned[-1] == text_that_got_returned[-2] == ".":
            if count % 40 == 0 and count != 0: print("\n")
            print("😶",end="")
            return get_length_to_cut(literature, count=count+1)


        # print(text_that_got_returned)
        length_to_cut = literature.find(text_that_got_returned)
        if length_to_cut == -1:
            if count % 40 == 0 and count != 0: print("\n")        
            print("😱",end="")
            return get_length_to_cut(literature, count=count+1)

        
        print(text_that_got_returned)
        length_to_cut += len(text_that_got_returned)
        break    

    return length_to_cut

In [8]:
chunks = []
window_size = 1000
while len(remaining_text) > 0:

    literature = remaining_text[:window_size]
    remaining_text = remaining_text[window_size:]
    
    length_to_cut = get_length_to_cut(literature)
    
    chunk = literature[:length_to_cut]
    remainder = literature[length_to_cut:]
    chunks.append(chunk)

    remaining_text = remainder + remaining_text

    write_text_to_file(chunk_directory, chunk_directory+number_to_four_digit_string(count), literature[:length_to_cut])
    count+=1
    # we're going to print the percent complete, to two decimal places
    percent_complete = 100 - (len(remaining_text)/len(full_text))*100
    percent_complete = round(percent_complete, 2)
    print(f"### Percent Complete: {percent_complete}%")

    

🤏🙅😱🤏🔚🙅😱😱🤏It's just because I do know you that I ask you to come to-morrow," said the girl, laughing.
Text has been written to whitenights/whitenights0029.txt
### Percent Complete: 19.34%
🙅🔚🙅🤏🔚😱😱🔚🙅🤏😱You will see ... the only thing is, I don't know how I am going to survive the next twenty-four hours.
Text has been written to whitenights/whitenights0030.txt
### Percent Complete: 20.11%
😱😶🤏🤏🙅😱🔚😱🤏🙅🤏🙅😱🔚🤏🙅😱🤏🔚😱🤏😱🤏😱Come, tell me aren't you glad that you were not angry and did not drive me away at the first moment, as any other woman would have done?
Text has been written to whitenights/whitenights0031.txt
### Percent Complete: 20.94%
🔚🙅🤏🙅😱🙅🙅🤏🤏🙅🙅🤏😶In two minutes you have made me happy for ever. Yes, happy; who knows, perhaps, you have reconciled me with myself, solved my doubts!... Perhaps such moments come upon me.... But there I will tell you all about it to-morrow, you shall know everything, everything.
Text has been written to whitenights/whitenights0032.txt
### Percent Complete: 21.19%
😱🙅🙅

In [9]:
# print(remaining_text)
literature.find("At this point Nastenka stopped and began laughing.")
print(literature)



My God, a whole moment of happiness! Is that too little for the whole of a man's life?
